In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from xgboost import XGBRegressor
from sklearn.model_selection import GroupShuffleSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import shap
import optuna
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

/Applications/miniconda3/envs/pitchml/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Read in formatted dataset
# assume repo root is two levels up from this notebook
ROOT = Path.cwd().parents[1]  # adjust if needed
DATA_DIR = ROOT / "data"

print(DATA_DIR)

print(ROOT)

df = pd.read_parquet(r"../../data/2023-03-30_2025-09-30_pitches_description_whiff_csw_stats_flattened.parquet")


/Users/matthewgillies/PitchArsenalModelling/data
/Users/matthewgillies/PitchArsenalModelling


In [3]:
df.head()

,pitcher,pitch_type,season,pitches,whiff_pct,csw_pct,p_throws,velo,VAA,HAA,...,self_usage,self_velo,self_HB,self_IVB,self_VAA,self_HAA,self_spin,self_ext,self_rel_x,self_rel_z
0,425794,CH,2023,89,0.100000,0.134831,R,81.431461,-4.546824,0.9675,...,0.049860,81.431461,-14.02382,8.755955,-4.546824,0.9675,1655.179775,6.511236,-1.335618,6.14236
1,425794,CU,2023,546,0.149378,0.252747,R,71.506044,-4.591665,1.387363,...,0.305882,71.506044,16.566374,-13.550989,-4.591665,1.387363,2647.908918,6.32334,-1.24359,6.284817
2,425794,FC,2023,404,0.146597,0.220297,R,82.852228,-4.121794,1.857984,...,0.226331,82.852228,5.577624,7.672574,-4.121794,1.857984,2321.953368,6.491688,-1.215272,6.287401
3,425794,FF,2023,178,0.100000,0.297753,R,85.73427,-3.745816,0.994451,...,0.099720,85.73427,-1.866742,14.824045,-3.745816,0.994451,2158.635294,6.530588,-1.232697,6.247135
4,425794,SI,2023,560,0.071429,0.260714,R,86.875,-4.051163,1.253781,...,0.313725,86.875,-13.252929,11.164071,-4.051163,1.253781,2167.058052,6.551685,-1.284518,6.134482


In [4]:
df.shape

(8397, 205)

In [5]:
df.isna().sum().sort_values(ascending=False)

SC_VAA        8390
SC_HAA        8390
SC_usage      8390
SC_velo       8390
SC_HB         8390
              ... 
IVB              0
pitch_type       0
has_CH           0
has_CS           0
self_rel_z       0
Length: 205, dtype: int64

In [6]:
# drop nas -> not a large proportion
# df = df.dropna()

In [7]:
df.shape

(8397, 205)

In [8]:
# drop duplicates in pitcher/season/pitch_type
df = df.drop_duplicates(subset=["pitcher", "season", "pitch_type"])
df.shape

(8397, 205)

In [9]:
# filter to FF only for initial analysis
ff_df = df[df["pitch_type"] == "SL"]

# filter to min 60 pitches thrown
ff_filtered = ff_df[ff_df['pitches'] > 60]

In [10]:
ff_df.head()

,pitcher,pitch_type,season,pitches,whiff_pct,csw_pct,p_throws,velo,VAA,HAA,...,self_usage,self_velo,self_HB,self_IVB,self_VAA,self_HAA,self_spin,self_ext,self_rel_x,self_rel_z
10,425844,SL,2023,444,0.217778,0.238739,R,79.16982,-4.399705,2.113677,...,0.200090,79.16982,13.576757,2.424865,-4.399705,2.113677,2464.178733,5.883559,-1.421194,6.155698
20,434378,SL,2023,709,0.267030,0.273625,R,86.877151,-5.255454,2.203544,...,0.254578,86.877151,4.515317,3.808181,-5.255454,2.203544,2513.184023,5.974612,-1.528209,6.974245
21,434378,SL,2024,305,0.202454,0.236066,R,86.699016,-5.364313,2.287399,...,0.193038,86.699016,4.001705,5.91423,-5.364313,2.287399,2441.642623,5.967541,-1.616951,7.069148
22,434378,SL,2025,623,0.307910,0.280899,R,87.108186,-5.443943,2.402287,...,0.231943,87.108186,4.06825,5.496886,-5.443943,2.402287,2496.504013,6.022953,-1.868491,7.065939
30,445276,SL,2023,86,0.375000,0.255814,R,84.139535,-4.846906,2.385485,...,0.119279,84.139535,7.240465,-3.795349,-4.846906,2.385485,2423.127907,6.774419,-2.413721,6.241047


In [11]:
ff_df.shape

(1386, 205)

In [12]:
# plot top 10 correlations with whiff_rate
correlations = ff_filtered.select_dtypes("number").corr()["whiff_pct"].abs().sort_values(ascending=False
)
top_10_features = correlations.index[1:11]  # exclude whiff_rate itself
print("Top 10 features correlated with whiff_rate:")
print(top_10_features)

Top 10 features correlated with whiff_rate:
Index(['CS_rel_z', 'CS_IVB', 'CS_velo', 'CS_spin', 'CS_HAA', 'CS_VAA',
       'CS_usage', 'CS_HB', 'CS_rel_x', 'whiff_pct'],
      dtype='object')


In [13]:
# create correlation heatmap for features with traget (whiff_pct)
# import seaborn as sns
# corr = ff_df.select_dtypes(include=[np.number]).corr()
# plt.figure(figsize=(24, 24))
# sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm")
# plt.title("Feature Correlation Heatmap")
# plt.show()

In [14]:
def objective(trial, X_train, y_train, X_valid, y_valid):
    model_type = trial.suggest_categorical(
        "model_type", ["xgboost", "lightgbm", "random_forest"]
    )

    if model_type == "xgboost":
        params = {
            "n_estimators": trial.suggest_int("xgb_n_estimators", 200, 2000),
            "max_depth": trial.suggest_int("xgb_max_depth", 3, 12),
            "learning_rate": trial.suggest_float("xgb_learning_rate", 0.01, 0.3, log=True),
            "subsample": trial.suggest_float("xgb_subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("xgb_colsample_bytree", 0.5, 1.0),
            "min_child_weight": trial.suggest_float("xgb_min_child_weight", 1e-3, 50.0, log=True),
            "gamma": trial.suggest_float("xgb_gamma", 0.0, 10.0),
            "reg_alpha": trial.suggest_float("xgb_reg_alpha", 1e-8, 10.0, log=True),
            "reg_lambda": trial.suggest_float("xgb_reg_lambda", 1e-8, 10.0, log=True),
            "random_state": 42,
            "n_jobs": -1,
        }
        model = XGBRegressor(**params)

    elif model_type == "lightgbm":
        params = {
            "n_estimators": trial.suggest_int("lgb_n_estimators", 200, 5000),
            "learning_rate": trial.suggest_float("lgb_learning_rate", 0.005, 0.3, log=True),
            "num_leaves": trial.suggest_int("lgb_num_leaves", 16, 512),
            "max_depth": trial.suggest_int("lgb_max_depth", -1, 16),
            "min_child_samples": trial.suggest_int("lgb_min_child_samples", 5, 200),
            "subsample": trial.suggest_float("lgb_subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("lgb_colsample_bytree", 0.5, 1.0),
            "reg_alpha": trial.suggest_float("lgb_reg_alpha", 1e-8, 10.0, log=True),
            "reg_lambda": trial.suggest_float("lgb_reg_lambda", 1e-8, 10.0, log=True),
            "random_state": 42,
            "n_jobs": -1,
            "verbose": -1,
        }
        model = LGBMRegressor(**params)

    else:  # random_forest
        params = {
            "n_estimators": trial.suggest_int("rf_n_estimators", 200, 2000),
            "max_depth": trial.suggest_int("rf_max_depth", 2, 50),
            "min_samples_split": trial.suggest_int("rf_min_samples_split", 2, 20),
            "min_samples_leaf": trial.suggest_int("rf_min_samples_leaf", 1, 20),
            "max_features": trial.suggest_categorical("rf_max_features", ["sqrt", "log2", 0.5, 1.0]),
            "bootstrap": trial.suggest_categorical("rf_bootstrap", [True, False]),
            "random_state": 42,
            "n_jobs": -1,
        }
        model = RandomForestRegressor(**params)

    # Fit
    if model_type == "catboost":
        model.fit(X_train, y_train, eval_set=(X_valid, y_valid), use_best_model=False)
    else:
        model.fit(X_train, y_train)

    # Evaluate
    y_pred = model.predict(X_valid)
    rmse = float(np.sqrt(mean_squared_error(y_valid, y_pred)))
    return rmse



In [15]:
# develop base xgboost workflow 
cat_cols = ['p_throws'] + [col for col in ff_df.columns if col.startswith('has_')]

target = "whiff_pct"

# 1) train/val split by pitcher - when we have multiple seasons with same pitches
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, val_idx = next(gss.split(ff_df, groups=ff_df["pitcher"]))

train = ff_df.iloc[train_idx].copy()
val   = ff_df.iloc[val_idx].copy()

# 2) choose features (exclude ids + targets)
drop_cols = ["pitcher", "season", "pitch_type", "whiff_pct", "csw_pct"]
X_train = train.drop(columns=[c for c in drop_cols if c in train.columns])
X_val   = val.drop(columns=[c for c in drop_cols if c in val.columns])

y_train = train[target].values
y_val   = val[target].values

# 3) one-hot categorical cols (simple + reliable)
X_train = pd.get_dummies(X_train, columns=cat_cols, dummy_na=True)
X_val   = pd.get_dummies(X_val, columns=cat_cols, dummy_na=True)

# align columns
X_train, X_val = X_train.align(X_val, join="left", axis=1, fill_value=0)

# 4) sample weights to reflect stability of rate targets
w_train = train["pitches"].values  # or np.sqrt(train["pitches"].values)



study = optuna.create_study(direction="minimize") 
study.optimize(lambda trial: objective(trial, X_train, y_train, X_val, y_val), n_trials=50)

print("Best trial:")
trial = study.best_trial
print("  RMSE: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


[I 2026-01-31 15:40:44,593] A new study created in memory with name: no-name-6018bfc1-1d73-48cd-91b8-d352bd013f4f
[I 2026-01-31 15:40:46,832] Trial 0 finished with value: 0.10123523680259168 and parameters: {'model_type': 'xgboost', 'xgb_n_estimators': 1986, 'xgb_max_depth': 6, 'xgb_learning_rate': 0.09146963706458094, 'xgb_subsample': 0.7044516850803688, 'xgb_colsample_bytree': 0.965969352523081, 'xgb_min_child_weight': 16.35073896298747, 'xgb_gamma': 5.073876609780468, 'xgb_reg_alpha': 0.1302033835042008, 'xgb_reg_lambda': 2.273499791709607e-06}. Best is trial 0 with value: 0.10123523680259168.
[I 2026-01-31 15:40:48,532] Trial 1 finished with value: 0.09456233530754014 and parameters: {'model_type': 'random_forest', 'rf_n_estimators': 1969, 'rf_max_depth': 23, 'rf_min_samples_split': 13, 'rf_min_samples_leaf': 15, 'rf_max_features': 'log2', 'rf_bootstrap': True}. Best is trial 1 with value: 0.09456233530754014.
[I 2026-01-31 15:40:49,241] Trial 2 finished with value: 0.0980617534260

Best trial:
  RMSE: 0.08561801866166258
  Params: 
    model_type: xgboost
    xgb_n_estimators: 1028
    xgb_max_depth: 3
    xgb_learning_rate: 0.22181329381345846
    xgb_subsample: 0.9171723775470515
    xgb_colsample_bytree: 0.5247740800966708
    xgb_min_child_weight: 0.802277705254542
    xgb_gamma: 0.06329215372665842
    xgb_reg_alpha: 0.03854423460191145
    xgb_reg_lambda: 0.0011305391139638033


In [16]:
trial.params

{'model_type': 'xgboost',
 'xgb_n_estimators': 1028,
 'xgb_max_depth': 3,
 'xgb_learning_rate': 0.22181329381345846,
 'xgb_subsample': 0.9171723775470515,
 'xgb_colsample_bytree': 0.5247740800966708,
 'xgb_min_child_weight': 0.802277705254542,
 'xgb_gamma': 0.06329215372665842,
 'xgb_reg_alpha': 0.03854423460191145,
 'xgb_reg_lambda': 0.0011305391139638033}

In [18]:
# define model with best params
best_params = trial.params
model = XGBRegressor(**best_params, random_state=42)

model.fit(X_train, y_train, sample_weight=w_train)

# train metrics:

train_pred = model.predict(X_train)
train_rmse = np.sqrt(mean_squared_error(y_train, train_pred))
train_r2 = r2_score(y_train, train_pred)
print("Train RMSE:", train_rmse, "R2:", train_r2)

pred = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, pred))
r2 = r2_score(y_val, pred)

print("RMSE:", rmse, "R2:", r2)

/Applications/miniconda3/envs/pitchml/lib/python3.10/site-packages/xgboost/training.py:199: UserWarning: [15:42:53] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:790: 
Parameters: { "model_type", "xgb_colsample_bytree", "xgb_gamma", "xgb_learning_rate", "xgb_max_depth", "xgb_min_child_weight", "xgb_n_estimators", "xgb_reg_alpha", "xgb_reg_lambda", "xgb_subsample" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Train RMSE: 0.00791496908829956 R2: 0.994496158382684
RMSE: 0.09242492233380555 R2: 0.16452621166998171


In [ ]:
# seaborn lmplot of predicted vs actual
# Build plotting dataframe
import seaborn as sns
plot_df = pd.DataFrame({
    "actual": y_val,
    "predicted": pred
})

# Plot
g = sns.lmplot(
    data=plot_df,
    x="actual",
    y="predicted",
    height=6,
    aspect=1,
    scatter_kws={"alpha": 0.6, "s": 40},
    line_kws={"color": "red"}
)

# Annotate R^2
plt.text(
    0.05, 0.95,
    f"$R^2 = {r2:.3f}$",
    transform=plt.gca().transAxes,
    fontsize=12,
    verticalalignment="top"
)

plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.title("Predicted vs Actual")

plt.show()



In [ ]:
# plot feature importance
importances = model.feature_importances_
feature_names = X_train.columns
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
}).sort_values(by='importance', ascending=False)
plt.figure(figsize=(10, 8))
sns.barplot(x='importance', y='feature', data=importance_df.head(40))
plt.title("Top 20 Feature Importances")
plt.show()

In [ ]:
# shap explainer and summary plot
explainer = shap.Explainer(model)
shap_values = explainer(X_val)
shap.summary_plot(shap_values, X_val)

# Workflow to look at individual player predictions

In [ ]:
from pybaseball import playerid_lookup

def get_mlbam_id(first: str, last: str) -> int:
    """
    Returns MLBAM id (key_mlbam) for a player name.
    If multiple results, returns the first row (you can disambiguate by birth year if needed).
    """
    res = playerid_lookup(last, first)
    if res.empty:
        raise ValueError(f"No player found for {first} {last}")

    # key_mlbam is what Statcast 'pitcher' column uses
    mlbam = res.iloc[0]["key_mlbam"]
    if pd.isna(mlbam):
        raise ValueError(f"Found player but no MLBAM id for {first} {last}")

    return int(mlbam)

# Example:
pitcher_id = get_mlbam_id("Gerrit", "Cole")
pitcher_id


In [ ]:
import pandas as pd

def preprocess_single_row(row_df, cat_cols, model_columns):
    """
    Preprocess a single-row dataframe to match X_train exactly.
    """
    X = row_df.copy()

    # ensure categoricals are strings and fill NA
    for c in cat_cols:
        if c in X.columns:
            X[c] = X[c].astype("string").fillna("NONE")

    # one-hot encode
    X = pd.get_dummies(X, columns=cat_cols, dummy_na=True)

    # align columns to training matrix
    X = X.reindex(columns=model_columns, fill_value=0)

    return X



In [ ]:
import shap
import matplotlib.pyplot as plt
from pybaseball import playerid_lookup

def explain_one_prediction(
    features_df,
    model,
    model_columns,
    cat_cols,
    first,
    last,
    season,
    pitch_type="FF"
):
    # --- lookup MLBAM id ---
    pid_df = playerid_lookup(last, first)
    if pid_df.empty:
        raise ValueError(f"No player found for {first} {last}")

    pitcher_id = int(pid_df.iloc[0]["key_mlbam"])

    # --- select row ---
    row = features_df[
        (features_df["pitcher"] == pitcher_id) &
        (features_df["season"] == season) &
        (features_df["pitch_type"] == pitch_type)
    ].copy()

    if row.empty:
        raise ValueError(f"No row for {first} {last}, {season}, {pitch_type}")

    # if somehow duplicated, take most-used
    if len(row) > 1:
        row = row.loc[[row["pitches"].idxmax()]]

    # --- drop non-features (exactly like training) ---
    drop_cols = ["pitcher", "season", "pitch_type", "whiff_pct", "csw_pct"]
    X_pre = row.drop(columns=[c for c in drop_cols if c in row.columns])

    # --- preprocess to match model ---
    X_row = preprocess_single_row(
        X_pre,
        cat_cols=cat_cols,
        model_columns=model_columns
    )

    # --- prediction ---
    pred = float(model.predict(X_row)[0])

    # --- SHAP ---
    explainer = shap.TreeExplainer(model)
    shap_values = explainer(X_row)

    # Waterfall plot (best for single prediction)
    shap.plots.waterfall(shap_values[0], max_display=20)

    return {
        "pitcher_id": pitcher_id,
        "season": season,
        "pitch_type": pitch_type,
        "prediction": pred,
        "raw_row": row,
        "X_row": X_row,
        "shap_values": shap_values
    }



In [ ]:
# Get pitchers from X_val:

val.head()

In [ ]:
# get playerr name from id: 
from pybaseball import playerid_reverse_lookup

# single id
for i in range(len(val)):
    print(val.iloc[i])
    print(playerid_reverse_lookup([val['pitcher'].iloc[i]], key_type="mlbam")[['name_first', 'name_last']])


In [ ]:
model_columns = X_train.columns

out = explain_one_prediction(
    features_df=df,   # your full season-level table
    model=model,
    model_columns=model_columns,
    cat_cols=cat_cols,
    first="Emmet",
    last="Sheehan",
    season=2025,
    pitch_type="SL"
)

print("Predicted SL whiff%:", out["prediction"])
out["raw_row"]
